## The aim of the project
...can be described on the example of airports. Imagine you have 10 cities, and for each pair of city, say A and B, there are ceratin number of people flying from A to B and another number of people flying from B to A. In total there would be 10 x 9 = 90 (quick math) possible routes and all are in demand. It would be a mess in the air, and this is only for 10 cities.

This issue can be resolved by promoting some of the cities to hubs. Flights between any two hubs are possible, but non-hub cities only have flights to hub(s) whcih they are allocated to. FYI, this actually happening in real world, that is why we cannot directly travel from Kazakhstan to USA, but have to stop somewhere in Europe, most probably in Frankfurt. If it is still not clear, here is a picture for help:

![title](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcS_J0OThbw4pDZzDspfH7tzFyjONHORv-gFCw&usqp=CAU)

On the figure there are 15 cities, 3 of them are hubs. All hubs are interconnected, while rest of cities are only connected to a ceratin hub. Other important point to mention is when a person is travelling between hubs it costs less than if the person would travel between same cities being not hubs. The explanation for this is the hubs are specially constructed and functioning for large flow of flights. Thus, their operation is optimized for large scales and they are operating more cost-efficiently. This is called the economy of scale. 

Hopefully, it is now more or less clear with airports. Let`s now shift to a general formulation and the task itself. In general problem cities are called nodes, hubs are hubs, when a node connected to a hubs it is called allocation to a hub, when a city promoted to a hub it is called location of hub, people travelling between cities are units. And economy of scale is taken into account as discount factor, a multiplier of cost of travel between hubs.

Almost there. There are plenty variants of hub location problem, but we are going to deal with the classic one, uncapacitated signle allocation p-hub location problem. Don`t be scared, it is just a name, the problem itself can be described by the following additional conditions:
* The number of hubs is fixed and predetermined
* Each non-hub node is allocated to only one hub
* There is no capacity limit for any hub (unlimited flow of flights through a hub s permitted)
* There is no fixed cost for hub location

So, all in all, we are given the demand of units to travel between each pair of nodes, the unit cost of any potential (even not used) flights between two nodes, the number of hubs and the discount factor. And with this we need to find the location of hubs and allocation of non-hub nodes to those hubs that minimizes the total cost.

Since this is NP-hard problem I have used metaheuristic approach to approximate the best solution. It would take too much time to explain the model, much more than a few paragraphs above. But if you are interested I have utilized Reduced Variable Neighborhood Search in combination with Tabu Search. The code with a small example is provided below.


In [1]:
class RVNS:
    random = __import__('random')
    time = __import__('time')
    
    def __init__(self, cost, flow, discount_factor, hub_n):
        self.cost = cost
        self.flow = flow
        
        self.node_n = len(flow)
        self.candidates = list(range(self.node_n))
            
        self.discount_factor = discount_factor
        self.hub_n = hub_n
        
        self.tabu = set()
#--------------------------------------------------------------------------------------------------------------------------
# Creating random solution
    def rand_sol(self):
        sel_candidates = self.random.sample(self.candidates, self.hub_n)       
        res = [sel_candidates[self.random.randint(0, self.hub_n - 1)] for _ in range(self.node_n)]

        for hub in sel_candidates:
            res[hub] = hub

        return res
#--------------------------------------------------------------------------------------------------------------------------
# Calculating the cost of random solution 
    def cost_calc(self, sol):
        res = 0

        for i in range(self.node_n):
            hub_i = sol[i]

            for j in range(i + 1, self.node_n):
                hub_j = sol[j]

                res += (self.cost[i][hub_i] + self.cost[hub_i][hub_j] * self.discount_factor + self.cost[hub_j][j]) * self.flow[i][j]
                res += (self.cost[j][hub_j] + self.cost[hub_j][hub_i] * self.discount_factor + self.cost[hub_i][i]) * self.flow[j][i]

        return res  
#-------------------------------------------------------------------------------------------------------------------------- 
# Random solution from 1st neighbourhood structure
    def neigh1(self, cur_sol, cur_cost):    
    #Creating a new solution:
        hubs = list(set(cur_sol))
        hub = self.random.choice(hubs)

        city_inds = []
        others = [] 
        for i in range(self.node_n):
            if i == hub:
                continue
            elif cur_sol[i] == hub:
                city_inds.append(i)
            else:
                others.append(i)

        if len(city_inds) == 0:
            return cur_sol, cur_cost

        ind = self.random.choice(city_inds)
        city_inds.append(hub)

        new_sol = cur_sol.copy()
        for j in city_inds:
            new_sol[j] = ind

    #Checking tabu list:
        str_sol = str(new_sol)
        if str_sol in self.tabu:
            return cur_sol, cur_cost
        else:
            self.tabu.add(str_sol)

    #Calculating the cost of a new solution:
        new_cost = cur_cost
        for j in city_inds:
            for i in others:
                B = new_sol[i]

                new_cost -= (self.cost[B][hub] * self.discount_factor + self.cost[hub][j]) * self.flow[i][j]
                new_cost -= (self.cost[j][hub] + self.cost[hub][B] * self.discount_factor) * self.flow[j][i]

                new_cost += (self.cost[B][ind] * self.discount_factor + self.cost[ind][j]) * self.flow[i][j]
                new_cost += (self.cost[j][ind] + self.cost[ind][B] * self.discount_factor) * self.flow[j][i]

            for i in city_inds:
                if i <= j:
                    continue
                new_cost += (self.cost[i][ind] + self.cost[ind][j] - self.cost[i][hub] - self.cost[hub][j]) * self.flow[i][j]
                new_cost += (self.cost[j][ind] + self.cost[ind][i] - self.cost[j][hub] - self.cost[hub][i]) * self.flow[j][i]

    #Outputting the best solution
        if new_cost < cur_cost:
            return new_sol, new_cost
        else:
            return cur_sol, cur_cost
#--------------------------------------------------------------------------------------------------------------------------      
# Random solution from 2nd neighbourhood
    def neigh2(self, cur_sol, cur_cost):
    #Creating a new solution:
        inds = [x for x in range(self.node_n) if cur_sol[x] != x]
        i = self.random.choice(inds)
        A = cur_sol[i]

        inds = [x for x in range(self.node_n) if cur_sol[x] != x and cur_sol[x] != A]
        if len(inds) == 0:
            return cur_sol, cur_cost

        j = self.random.choice(inds)
        B = cur_sol[j]

        new_sol = cur_sol.copy()
        new_sol[i], new_sol[j] = B, A

    #Checking tabu list:
        str_sol = str(new_sol)
        if str_sol in self.tabu:
            return cur_sol, cur_cost
        else:
            self.tabu.add(str_sol)

    #Calculating the cost of a new solution:
        new_cost = cur_cost
        for k in range(self.node_n):
            if k == i or k == j:
                continue

            C = new_sol[k]
            new_cost += (self.cost[C][B] * self.discount_factor + self.cost[B][i] - self.cost[C][A] * self.discount_factor - self.cost[A][i]) * self.flow[k][i]
            new_cost += (self.cost[i][B] + self.cost[B][C] * self.discount_factor - self.cost[i][A] - self.cost[A][C] * self.discount_factor) * self.flow[i][k]
            new_cost += (self.cost[C][A] * self.discount_factor + self.cost[A][j] - self.cost[C][B] * self.discount_factor - self.cost[B][j]) * self.flow[k][j]
            new_cost += (self.cost[j][A] + self.cost[A][C] * self.discount_factor - self.cost[j][B] - self.cost[B][C] * self.discount_factor) * self.flow[j][k]

        new_cost += (self.cost[i][B] + self.cost[B][A] * self.discount_factor + self.cost[A][j] - self.cost[i][A] - self.cost[A][B] * self.discount_factor - self.cost[B][j]) * self.flow[i][j]
        new_cost += (self.cost[j][A] + self.cost[A][B] * self.discount_factor + self.cost[B][i] - self.cost[j][B] - self.cost[B][A] * self.discount_factor - self.cost[A][i]) * self.flow[j][i]

    #Outputting the best solution
        if new_cost < cur_cost:
            return new_sol, new_cost
        else:
            return cur_sol, cur_cost
#--------------------------------------------------------------------------------------------------------------------------      
# Random solution from 3rd neighbourhood
    def neigh3(self, cur_sol, cur_cost):
    #Creating a new solution:
        inds = [x for x in range(self.node_n) if cur_sol[x] != x]
        i = self.random.choice(inds)
        hub_i = cur_sol[i]

        hubs = list(set(cur_sol))
        hubs.remove(hub_i)
        hub = self.random.choice(hubs)

        new_sol = cur_sol.copy()
        new_sol[i] = hub

    #Checking tabu list:
        str_sol = str(new_sol)
        if str_sol in self.tabu:
            return cur_sol, cur_cost
        else:
            self.tabu.add(str_sol)

    #Calculating the cost of a new solution:
        new_cost = cur_cost
        for j in self.candidates:
            hub_j = new_sol[j]
            new_cost -= (self.cost[i][hub_i] + self.cost[hub_i][hub_j] * self.discount_factor) * self.flow[i][j]
            new_cost -= (self.cost[hub_j][hub_i] * self.discount_factor + self.cost[hub_i][i]) * self.flow[j][i]

            new_cost += (self.cost[hub_j][hub] * self.discount_factor + self.cost[hub][i]) * self.flow[j][i]
            new_cost += (self.cost[i][hub] + self.cost[hub][hub_j] * self.discount_factor) * self.flow[i][j]

    #Outputting the best solution
        if new_cost < cur_cost:
            return new_sol, new_cost
        else:
            return cur_sol, cur_cost
#--------------------------------------------------------------------------------------------------------------------------      
# Random solution from 4th neighbourhood
    def neigh4(self, cur_sol, cur_cost):
    #Creating a new solution:
        hubs = list(set(cur_sol))
        hub_i = self.random.choice(hubs)

        hubs.remove(hub_i)
        hub_j = self.random.choice(hubs)

        new_sol = cur_sol.copy()
        for ind, city in enumerate(new_sol):
            if ind == hub_i or ind == hub_j:
                continue

            if city == hub_i:
                new_sol[ind] = hub_j

            if city == hub_j:
                new_sol[ind] = hub_i

    #Checking tabu list:
        str_sol = str(new_sol)
        if str_sol in self.tabu:
            return cur_sol, cur_cost
        else:
            self.tabu.add(str_sol)

    #Calculating the cost of a new solution:
        new_cost = self.cost_calc(new_sol)

    #Outputting the best solution
        if new_cost < cur_cost:
            return new_sol, new_cost
        else:
            return cur_sol, cur_cost
#--------------------------------------------------------------------------------------------------------------------------      
# Running algorithm
    def run(self, ini_sol, iter_n):
        k = 1
        report_sol = ini_sol.copy()
        report_cost = self.cost_calc(report_sol)
        
        while iter_n:
            iter_n -= 1
            if k == 1:
                report_sol, report_cost = self.neigh1(report_sol, report_cost)

            elif k == 2:
                report_sol, report_cost = self.neigh2(report_sol, report_cost)
        
            elif k == 3:
                report_sol, report_cost = self.neigh3(report_sol, report_cost)
               
            else:
                report_sol, report_cost = self.neigh4(report_sol, report_cost)
                
            k = k % 4 + 1
        return report_sol, report_cost
#--------------------------------------------------------------------------------------------------------------------------      
# Multiple runs of algorithm
    def multiple_run(self, count, iter_n):
        start = self.time.time()
        best_sol = None
        best_cost = float('Inf')
        total_cost = 0
        
        for _ in range(count):
            ini_sol = self.rand_sol()
            cur_sol, cur_cost = self.run(ini_sol, iter_n)
            total_cost += cur_cost

            if cur_cost < best_cost:
                best_sol = cur_sol
                best_cost = cur_cost

        end = self.time.time()
        print(f'Best solution is:\n{best_sol}\n')
        print(f'Best solution hubs: {sorted([x + 1 for x in set(best_sol)])}')
        print(f'with the best score of: {best_cost}\n')
        print(f'Average score was {total_cost / count}\n')
        print(f'Average time per run was {(end - start) / count}\n')
        print(f'Size of tabu list is: {len(self.tabu)}')

The cells below are just an example of implementation of the algorithm. Please, feel free to use your other datasets (they are larger) I have provided in the **Data** folder. Alternatively you can also use your own data, just make sure that the you are using NxN matrix for both flow and cost inputs for **RVNS** function. Also, you can variate number of runs of algorithm and number of iterations per run, for large datasets it is recommended to use 10 runs with 10000 iterations. Enjoy!

In [2]:
# please upload the data for cost and flow (example of 10 nodes provided)
import numpy as np
cost = np.loadtxt(open('Data/10cost.csv', "rb"), delimiter=",", skiprows=1)
flow = np.loadtxt(open('Data/10flow.csv', "rb"), delimiter=",", skiprows=1)

# please indicate number of hubs, discount factor (example of 3 hubs with 0.2 df provided)
hubs_number = 3
discount = 0.2
# please select number of runs and number of iterations per one run
# (100 runs with 1000 iterations were found optimum for this dataset)
runs_number = 100
iterations_per_run = 1000

In [3]:
# running the algorithm
solution = RVNS(cost, flow, discount, hubs_number)
solution.multiple_run(runs_number, iterations_per_run)

Best solution is:
[5, 5, 5, 3, 5, 5, 6, 6, 5, 6]

Best solution hubs: [4, 6, 7]
with the best score of: 491.9343312144027

Average score was 613.0126592128938

Average time per run was 0.016192348003387452

Size of tabu list is: 8248
